Python for ML
ML libraries built for Python:

1. Numpy efficiently load large dataset large memory
2. SciPy: computing features/ numeric optimization features to calculate recommended products for users
3. Pandas: represent data as spreadsheet (edit and perform calculations)

In [5]:
# Import Libraries
import os
import pandas as pd
import numpy as np

In [25]:
# Get  current working directory
project_root = os.getcwd()
datasets = os.path.join(project_root, "datasets/")

# Define the dtype for column 1 to avoid mixed types
dtype_dict = {1: str}

# load the Data
product_info = pd.read_csv(datasets + "product_info.csv", dtype=dtype_dict)
reviews_250 = pd.read_csv(datasets + "reviews_0-250.csv", dtype=dtype_dict)
reviews_500 = pd.read_csv(datasets + "reviews_250-500.csv", dtype=dtype_dict)
reviews_750 = pd.read_csv(datasets + "reviews_500-750.csv", dtype=dtype_dict)
reviews_1250 = pd.read_csv(datasets + "reviews_750-1250.csv", dtype=dtype_dict)
reviews_end = pd.read_csv(datasets + "reviews_1250-end.csv", dtype=dtype_dict)

# Review Contents
print("product_info shape:", product_info.shape)
print("reviews_250 shape:", reviews_250.shape)
print("reviews_500 shape:", reviews_500.shape)
print("reviews_750 shape:", reviews_750.shape)
print("reviews_1250 shape:", reviews_1250.shape)
print("reviews_end shape:", reviews_end.shape)

product_info shape: (8494, 27)
reviews_250 shape: (602130, 19)
reviews_500 shape: (206725, 19)
reviews_750 shape: (116262, 19)
reviews_1250 shape: (119317, 19)
reviews_end shape: (49977, 19)


In [26]:
# Combine Reviews - MIXED TYPES WARNING
all_reviews = pd.concat([
    reviews_250, reviews_500, reviews_750, 
    reviews_1250, reviews_end], ignore_index=True)
all_reviews.head()

,Unnamed: 0,author_id,rating,is_recommended,helpfulness,total_feedback_count,total_neg_feedback_count,total_pos_feedback_count,submission_time,review_text,review_title,skin_tone,eye_color,skin_type,hair_color,product_id,product_name,brand_name,price_usd
0,0,1741593524,5,1.0,1.0,2,0,2,2023-02-01,I use this with the Nudestix “Citrus Clean Bal...,Taught me how to double cleanse!,NaN,brown,dry,black,P504322,Gentle Hydra-Gel Face Cleanser,NUDESTIX,19.0
1,1,31423088263,1,0.0,NaN,0,0,0,2023-03-21,I bought this lip mask after reading the revie...,Disappointed,NaN,NaN,NaN,NaN,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0
2,2,5061282401,5,1.0,NaN,0,0,0,2023-03-21,My review title says it all! I get so excited ...,New Favorite Routine,light,brown,dry,blonde,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0
3,3,6083038851,5,1.0,NaN,0,0,0,2023-03-20,I’ve always loved this formula for a long time...,Can't go wrong with any of them,NaN,brown,combination,black,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0
4,4,47056667835,5,1.0,NaN,0,0,0,2023-03-20,"If you have dry cracked lips, this is a must h...",A must have !!!,light,hazel,combination,NaN,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0


In [20]:
product_info.head()

,product_id,product_name,brand_id,brand_name,loves_count,rating,reviews,size,variation_type,variation_value,...,online_only,out_of_stock,sephora_exclusive,highlights,primary_category,secondary_category,tertiary_category,child_count,child_max_price,child_min_price
0,P473671,Fragrance Discovery Set,6342,19-69,6320,3.6364,11.0,NaN,NaN,NaN,...,1,0,0,"['Unisex/ Genderless Scent', 'Warm &Spicy Scen...",Fragrance,Value & Gift Sets,Perfume Gift Sets,0,NaN,NaN
1,P473668,La Habana Eau de Parfum,6342,19-69,3827,4.1538,13.0,3.4 oz/ 100 mL,Size + Concentration + Formulation,3.4 oz/ 100 mL,...,1,0,0,"['Unisex/ Genderless Scent', 'Layerable Scent'...",Fragrance,Women,Perfume,2,85.0,30.0
2,P473662,Rainbow Bar Eau de Parfum,6342,19-69,3253,4.2500,16.0,3.4 oz/ 100 mL,Size + Concentration + Formulation,3.4 oz/ 100 mL,...,1,0,0,"['Unisex/ Genderless Scent', 'Layerable Scent'...",Fragrance,Women,Perfume,2,75.0,30.0
3,P473660,Kasbah Eau de Parfum,6342,19-69,3018,4.4762,21.0,3.4 oz/ 100 mL,Size + Concentration + Formulation,3.4 oz/ 100 mL,...,1,0,0,"['Unisex/ Genderless Scent', 'Layerable Scent'...",Fragrance,Women,Perfume,2,75.0,30.0
4,P473658,Purple Haze Eau de Parfum,6342,19-69,2691,3.2308,13.0,3.4 oz/ 100 mL,Size + Concentration + Formulation,3.4 oz/ 100 mL,...,1,0,0,"['Unisex/ Genderless Scent', 'Layerable Scent'...",Fragrance,Women,Perfume,2,75.0,30.0


In [28]:
all_reviews.dtypes

Unnamed: 0                    int64
author_id                    object
rating                        int64
is_recommended              float64
helpfulness                 float64
total_feedback_count          int64
total_neg_feedback_count      int64
total_pos_feedback_count      int64
submission_time              object
review_text                  object
review_title                 object
skin_tone                    object
eye_color                    object
skin_type                    object
hair_color                   object
product_id                   object
product_name                 object
brand_name                   object
price_usd                   float64
dtype: object